In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu copy


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['routeLength'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_emissions(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_emissions(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(200)
train_Y = user_function_emissions(train_X)

[[ 5.         10.2172657  13.88963595 51.0735286   3.          4.32014929]
 [ 3.         13.43993147 15.0250675  30.206108    1.          4.52868338]
 [ 5.         18.20827673 15.09092485 56.83776676  1.          1.68020994]
 [12.         22.00282661  9.8854464  64.4366358   1.          2.44239115]
 [ 3.         20.7372536   7.46117622 46.88339848  2.          2.43544337]
 [19.         13.7188585  22.52773254 68.05705021  2.          3.52060857]
 [11.         10.33651549 11.45189327 64.79167003  3.          2.00011697]
 [16.         20.63617127 22.4911205  41.59620486  3.          3.72480387]
 [15.         24.28949627 21.50178834 58.53323519  2.          3.45809672]
 [11.         18.24192225 16.58506508 43.027893    1.          4.34989808]
 [17.         12.20873058  6.32144876 57.04205778  1.          3.79052292]
 [15.         13.2874162  10.62669    66.40639779  3.          1.75936091]
 [13.         16.54011184  6.37259335 36.42997895  1.          2.74715757]
 [ 7.         16.6629025 


Evaluating input: 36 of 50

 Retrying in 1 seconds

Evaluating input: 37 of 50

 Retrying in 1 seconds

Evaluating input: 38 of 50

 Retrying in 1 seconds

Evaluating input: 39 of 50

 Retrying in 1 seconds

Evaluating input: 40 of 50

 Retrying in 1 seconds

Evaluating input: 41 of 50

 Retrying in 1 seconds

Evaluating input: 42 of 50

 Retrying in 1 seconds

Evaluating input: 43 of 50

 Retrying in 1 seconds

Evaluating input: 44 of 50

 Retrying in 1 seconds

Evaluating input: 45 of 50

 Retrying in 1 seconds

Evaluating input: 46 of 50

 Retrying in 1 seconds

Evaluating input: 47 of 50

 Retrying in 1 seconds

Evaluating input: 48 of 50

 Retrying in 1 seconds

Evaluating input: 49 of 50

 Retrying in 1 seconds

Evaluating input: 50 of 50

 Retrying in 1 seconds
[[13.         16.38771605  9.1137267  34.3371737   1.          3.32786953]
 [ 7.         14.13088014  6.55044648 66.23434601  2.          4.59401489]
 [16.         24.8626929  21.52485166 64.63365442  2.          2.34276


Evaluating input: 131 of 200

 Retrying in 1 seconds

Evaluating input: 132 of 200

 Retrying in 1 seconds

Evaluating input: 133 of 200

 Retrying in 1 seconds

Evaluating input: 134 of 200

 Retrying in 1 seconds

Evaluating input: 135 of 200

 Retrying in 1 seconds

Evaluating input: 136 of 200

 Retrying in 1 seconds

Evaluating input: 137 of 200

 Retrying in 1 seconds

Evaluating input: 138 of 200

 Retrying in 1 seconds

Evaluating input: 139 of 200

 Retrying in 1 seconds

Evaluating input: 140 of 200

 Retrying in 1 seconds

Evaluating input: 141 of 200

 Retrying in 1 seconds

Evaluating input: 142 of 200

 Retrying in 1 seconds

Evaluating input: 143 of 200

 Retrying in 1 seconds

Evaluating input: 144 of 200

 Retrying in 1 seconds

Evaluating input: 145 of 200

 Retrying in 1 seconds

Evaluating input: 146 of 200

 Retrying in 1 seconds

Evaluating input: 147 of 200

 Retrying in 1 seconds

Evaluating input: 148 of 200

 Retrying in 1 seconds

Evaluating input: 149 of 20

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [8]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = 81.62241621589436
Optimization restart 2/5, f = 81.62241640537142
Optimization restart 3/5, f = 65.19865052757268
Optimization restart 4/5, f = 81.62241639425167
Optimization restart 5/5, f = 81.62241632460682


GP_regression.,value,constraints,priors
rbf.variance,259.7135845416435,+ve,
rbf.lengthscale,17.238192063611056,+ve,
Gaussian_noise.variance,1.2484806063965144e-21,+ve,


#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
)

In [12]:
ed_loop.run_loop(user_function_emissions, 200)

Optimization restart 1/5, f = 65.19865052757268
Optimization restart 2/5, f = 81.62241617023763
Optimization restart 3/5, f = 81.62241627852042
Optimization restart 4/5, f = 81.62241628503907
Optimization restart 5/5, f = 81.62241632067877
[[20.          8.         27.         44.28235931  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 68.37136365481754
Optimization restart 2/5, f = 85.21182644010432
Optimization restart 3/5, f = 85.21182643982924
Optimization restart 4/5, f = 85.21182643935616
Optimization restart 5/5, f = 85.21182645045944
[[ 3.  25.  27.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 71.42316406427614
Optimization restart 2/5, f = 90.54448491785342
Optimization restart 3/5, f = 71.4231640646276
Optimization restart 4/5, f = 71.42316407114464
Opt

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = 122.83771917445281
Optimization restart 2/5, f = 151.34722568032518
Optimization restart 3/5, f = 151.34722568030327
Optimization restart 4/5, f = 151.34722505501261
Optimization restart 5/5, f = 151.3472256751156
[[11.          8.         18.47115622 30.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 131.0036478470794
Optimization restart 2/5, f = 154.96818590648473
Optimization restart 3/5, f = 154.9681851919844
Optimization restart 4/5, f = 154.96818589341075
Optimization restart 5/5, f = 154.96818589241025
[[20. 25. 27. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 134.05199148476208
Optimization restart 2/5, f = 158.56109905045778
Optimization restart 3/5, f = 158.5610990504527
Optimization restart 4/5

 Retrying in 1 seconds
Optimization restart 1/5, f = 178.6178384946441
Optimization restart 2/5, f = 178.61783809604188
Optimization restart 3/5, f = 178.61783830116462
Optimization restart 4/5, f = 178.61783827054037
Optimization restart 5/5, f = 224.01287135993596
[[ 3.         13.5332352  16.12545717 70.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 181.3264346058295
Optimization restart 2/5, f = 181.3263673111433
Optimization restart 3/5, f = 228.0763435738092
Optimization restart 4/5, f = 228.07634641394512
Optimization restart 5/5, f = 228.07634692081206
[[ 3.          8.         17.48727488 40.04921579  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 184.05471907753324
Optimization restart 2/5, f = 184.05471639474996
Optimization restart 3/5,

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


...

Optimization restart 1/5, f = 559.7008027769027
Optimization restart 2/5, f = 856.6389536279062
Optimization restart 3/5, f = 559.7007618010415
Optimization restart 4/5, f = 559.7007617912623
Optimization restart 5/5, f = 559.7007617473279
[[20.          8.          5.         39.51067929  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 561.4066587555121
Optimization restart 2/5, f = 561.4066550682019
Optimization restart 3/5, f = 561.4066550681638
Optimization restart 4/5, f = 561.4066550681694
Optimization restart 5/5, f = 561.4066550681683
[[ 3.         25.          5.         39.25017065  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 563.3137680530795
Optimization restart 2/5, f = 563.3133807133086
Optimization restart 3/5, f = 563.3133806856165
Optimization restart 4/5, f = 563.3133806955581
Optimization restart 5/5, f = 563.3133806856266


### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [13]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = 81.62241621589436
Optimization restart 2/5, f = 81.62241630541614
Optimization restart 3/5, f = 81.62241624290738
Optimization restart 4/5, f = 81.62241631238274
Optimization restart 5/5, f = 81.62241627936152


#### Acquisition Function

In [14]:
integrated_variance_reduction = IntegratedVarianceReduction(model=emukit_model_ird, space=parameter_space)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
)

In [17]:
ed_loop.run_loop(user_function_emissions, 200)

Optimization restart 1/5, f = 81.62241621589436
Optimization restart 2/5, f = 81.62241633007368
Optimization restart 3/5, f = 81.62241627922751
Optimization restart 4/5, f = 81.62241626065816
Optimization restart 5/5, f = 81.62241630983371
[[ 7.         18.12328781 19.96482792 31.99627141  2.          3.98846504]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 86.5696811768876
Optimization restart 2/5, f = 86.56968126976473
Optimization restart 3/5, f = 86.56968129385525
Optimization restart 4/5, f = 69.79864087112088
Optimization restart 5/5, f = 86.56968126984441
[[15.         11.83289775 19.62717668 43.46431949  2.          3.32225208]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 73.24614467600053
Optimization restart 2/5, f = 90.26387470762349
Optimization restart 3/5, f = 90.26387471043343
Optimizat

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = 85.57988157623357
Optimization restart 5/5, f = 85.57988157612115
[[14.          9.85085269 13.58773724 63.60812301  3.          2.64488865]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 86.73245614027624
Optimization restart 2/5, f = 86.83882732116555
Optimization restart 3/5, f = 86.83882732118516
Optimization restart 4/5, f = 86.83882732138974
Optimization restart 5/5, f = 86.83882732122638
[[16.         22.312445   12.25767353 35.26393351  2.          3.07454595]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 89.83472182093655
Optimization restart 2/5, f = 89.83472182121041
Optimization restart 3/5, f = 90.40473527620259
Optimization restart 4/5, f = 90.404735276235


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 90.40473527630742
[[11.         15.09482989 14.91290636 41.41238274  1.          2.05709556]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 91.03824671464872
Optimization restart 2/5, f = 91.67675321234357
Optimization restart 3/5, f = 91.6767532122993
Optimization restart 4/5, f = 91.67675321227611
Optimization restart 5/5, f = 91.67675321256016
[[17.         13.40125219 22.21099984 56.41904873  2.          2.6760188 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = 92.83541061043293
Optimization restart 2/5, f = 92.83541061035217
Optimization restart 3/5, f = 93.80414692960358
Optimization restart 4/5, f = 93.80414693017153
Optimization restart 5/5, f = 93.8041469296348
[[15.         12.63774591 13.76851391 34.16241     2.          2.96934045]]

User function time loss calle

...

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 247.30279079746037
Optimization restart 2/5, f = 247.29651337643088
Optimization restart 3/5, f = 247.29651284444122
Optimization restart 4/5, f = 247.2965128450411
Optimization restart 5/5, f = 247.2965132283425
[[18.         25.         22.36772528 49.19431291  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 249.0462443071689
Optimization restart 2/5, f = 249.04511924882024
Optimization restart 3/5, f = 249.04511924911395
Optimization restart 4/5, f = 249.04511924882686
Optimization restart 5/5, f = 249.04511924897702


KeyboardInterrupt: 

### Random GP

Supervised learning

In [18]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y, noise_var=1e-10)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=200)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/200, f = 545.4165292495991
Optimization restart 2/200, f = 434.5785373411501
Optimization restart 3/200, f = 434.5785373415936
Optimization restart 4/200, f = 434.57853734117805
Optimization restart 5/200, f = 434.57853736345714
Optimization restart 6/200, f = 434.57853734116526
Optimization restart 7/200, f = 434.578537342638
Optimization restart 8/200, f = 434.5785374008318
Optimization restart 9/200, f = 434.5785373420476
Optimization restart 10/200, f = 434.5785373413296
Optimization restart 11/200, f = 434.5785391893328
Optimization restart 12/200, f = 434.57853734103753
Optimization restart 13/200, f = 434.57853734105555
Optimization restart 14/200, f = 434.5785373496638
Optimization restart 15/200, f = 434.5785373411432
Optimization restart 16/200, f = 434.5785373654532
Optimization restart 17/200, f = 434.5785373411885
Optimization restart 18/200, f = 434.57853734112
Optimization restart 19/200, f = 434.57853734107437
Optimization restart 20/200, f = 434.

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 39/200, f = 436.57883152210394
Optimization restart 40/200, f = 606.0558627473357
Optimization restart 41/200, f = 434.57853734528175
Optimization restart 42/200, f = 434.57853734130066
Optimization restart 43/200, f = 434.5785373518034
Optimization restart 44/200, f = 434.5785373410467
Optimization restart 45/200, f = 434.57853734107977
Optimization restart 46/200, f = 434.5785373416029
Optimization restart 47/200, f = 434.57853734154827
Optimization restart 48/200, f = 756.311359316205
Optimization restart 49/200, f = 434.5785373412696
Optimization restart 50/200, f = 434.5785373414483
Optimization restart 51/200, f = 434.57853734122693
Optimization restart 52/200, f = 434.57853734748744
Optimization restart 53/200, f = 756.3113582369024
Optimization restart 54/200, f = 434.57853763374777
Optimization restart 55/200, f = 434.5785373411916
Optimization restart 56/200, f = 434.5785377250106
Optimization restart 57/200, f = 434.5785373411044
Optimization restart 58/

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 196/200, f = 434.578538341205
Optimization restart 197/200, f = 434.57853734106214
Optimization restart 198/200, f = 434.5785373418
Optimization restart 199/200, f = 434.5785373410893
Optimization restart 200/200, f = 756.3113614252726


GP_regression.,value,constraints,priors
rbf.variance,1322.3545038473712,+ve,
rbf.lengthscale,30.061839854705823,+ve,
Gaussian_noise.variance,2.0435415133431376,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [19]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  1.3953025994048731


In [20]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  1.7167904663824762


In [21]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  1.3678332715819297


In [23]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is -2.008% better than training on random points.'

In [24]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is -25.512% better than training on random points.'

### Save Models

In [26]:
with open(f'experimental_design/models/experimental_design_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)
        
with open(f'experimental_design/models/experimental_design_integrated_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design/models/270_random_points_supervised_GP_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)